In [1]:
%%capture --no-stderr
%pip install --quiet -U langgraph langchain-google-genai tavily-python langchain-tavily chromadb

In [31]:
recentDiscordMessages = [
    "단아히: ㅜㅜㅜㅜㅜ",
    "단아히: 졸려",
    "단아히: 4시간자고 일어나",
    "단아히: 다음은 금당이야",
    "ㅇㅎㅅ: dnehdqkqh dne 우동바보",
    "ㅇㅎㅅ: 당직이나 설",
    "단아히: 준내 난 왜 맨나갇새대9ㄱ9새세세덱ㄹ이니집ㅈㅅㅅㄱ2ㅅ9939ㄱ939292010101010101010100101010101100011",
    "단아히: 오늘 뉴로롱 안만들고 잤어",
    "단아히: 개인정비때 걍 자버림",
    "단아히: 너무 아깝다",
    "단아히: 씻지도 않았어",
    "단아히: 지금 씻어야징",
    "ㅇㅎㅅ: 스팸 혼자 남자마자",
    "ㅇㅎㅅ: 바로 잠수네",
    "ㅇㅎㅅ: 화가치밀어 오른다아아아ㅏㅏㅏㅏ",
    "ㅇㅎㅅ: 김현빈 이자식",
    "ㅇㅎㅅ: 창성이 알려줄려고",
    "ㅇㅎㅅ: 일부러 죽었네",
    "ㅇㅎㅅ: 화가",
    "ㅇㅎㅅ: 더 치밀어",
    "ㅇㅎㅅ: 올느다ㅏㅏㅏㅏㅏㅏㅏㅏㅏ",
    "ㅇㅎㅅ: :emoji_8~1:",
    "단아히: 우소자나이",
    "단아히: 노래진짜좋습니다",
    "단아히: 들으세요",
    "단아히: 숙제입니다",
    "bp: ㅋㅋㅋㅋㅋ",
    "스팸: 그렇게",
    "스팸: 우동은",
    "스팸: 죽었다",
    "스팸: 갑자기 겜 팅겨버림",
    "스팸: @ㅇㅎㅅ",
    "bp: ㄷㄷ",
    "bp: 우리 머치",
    "bp: 한판하고 초대해줄게",
]

In [2]:
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGSMITH_PROJECT"] = "pr-granular-trinket-22"

In [6]:
from pydantic import BaseModel, Field
from typing import List

class TopicCluster(BaseModel):
    """A cluster representing a single topic or thread found in the conversation."""
    
    representative_summary: str = Field(
        description="A single, concise sentence summarizing the main topic and outcome of this cluster."
    )
    
    messages_included: List[str] = Field(
        description="A list of the original messages (sender: content) that belong to this topic."
    )

class ConversationClusters(BaseModel):
    """The final structured output containing all identified topic clusters."""
    
    clusters: List[TopicCluster]
    
    all_messages_accounted_for: bool = Field(
        description="MUST be set to 'True'. This confirms that every single message from the input conversation_string has been placed into one of the TopicCluster 'messages_included' lists."
    )

In [7]:
import os
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate

cluster_generator_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """### SYSTEM ROLE: CONVERSATION CLUSTERER
You are a Conversation Analyst. Your task is to segment the full conversation history provided below into distinct topic threads (clusters).

### INSTRUCTIONS
1. Identify all primary subjects (e.g., Personal Status, Game Discussion, Recommendations).
2. For each cluster, create a **single, concise sentence** that acts as the **representative_summary**.
3. Include the original messages that belong to that topic thread in the 'messages_included' list.
4. Output ONLY the structured JSON object."""
        ),
        ("placeholder", "{raw_messages}")
    ]
)

load_dotenv()
cluster_generator = cluster_generator_prompt | ChatGoogleGenerativeAI(
    model="gemini-2.0-flash", google_api_key=os.getenv("GEMINI_API_KEY"), temperature=0 
).with_structured_output(ConversationClusters)

In [8]:
cluster_generator.invoke(
    {
        "raw_messages": [
            ("user", full_conversation_string),
        ]
    }
)

ConversationClusters(clusters=[TopicCluster(representative_summary='단아히 expresses tiredness and discusses their sleep schedule and personal hygiene routines.', messages_included=['단아히: ㅜㅜㅜㅜㅜ', '단아히: 졸려', '단아히: 4시간자고 일어나', '단아히: 다음은 금당이야', '단아히: 준내 난 왜 맨나갇새대9ㄱ9새세세덱ㄹ이니집ㅈㅅㅅㄱ2ㅅ9939ㄱ939292010101010101010100101010101100011', '단아히: 오늘 뉴로롱 안만들고 잤어', '단아히: 개인정비때 걍 자버림', '단아히: 너무 아깝다', '단아히: 씻지도 않았어', '단아히: 지금 씻어야징']), TopicCluster(representative_summary="ㅇㅎㅅ expresses frustration and anger related to a game, possibly due to a teammate's actions.", messages_included=['ㅇㅎㅅ: dnehdqkqh dne 우동바보', 'ㅇㅎㅅ: 당직이나 설', 'ㅇㅎㅅ: 스팸 혼자 남자마자', 'ㅇㅎㅅ: 바로 잠수네', 'ㅇㅎㅅ: 화가치밀어 오른다아아아ㅏㅏㅏㅏ', 'ㅇㅎㅅ: 김현빈 이자식', 'ㅇㅎㅅ: 창성이 알려줄려고', 'ㅇㅎㅅ: 일부러 죽었네', 'ㅇㅎㅅ: 화가', 'ㅇㅎㅅ: 더 치밀어', 'ㅇㅎㅅ: 올느다ㅏㅏㅏㅏㅏㅏㅏㅏㅏ', 'ㅇㅎㅅ: :emoji_8~1:']), TopicCluster(representative_summary='단아히 recommends a song and assigns it as homework.', messages_included=['단아히: 우소자나이', '단아히: 노래진짜좋습니다', '단아히: 들으세요', '단아히: 숙제입니다']), TopicCluster(representative_summary='스팸 recounts 

In [36]:
from pydantic import BaseModel, Field
from typing import List
from enum import Enum 

class ValidityType(str, Enum):
    SHORT_TERM = "SHORT_TERM" # 일시적인 상태 (예: 피로, 오늘 한 일)
    LONG_TERM = "LONG_TERM" # 지속적인 취향/사실 (예: 좋아하는 노래, 습관)

class UserObservationItem(BaseModel):
    """A single, meaningful observation about a user derived from the conversation."""
    user: str = Field(description="The nickname of the user to whom the observation pertains (e.g., 단아히, ㅇㅎㅅ, bp).")
    observation_summary: str = Field(description="A single, concise sentence summarizing a key fact, status, or preference about the user, without including temporal phrases.")
    validity_type: ValidityType = Field(description="The persistence of the observation, categorized as SHORT_TERM (transient) or LONG_TERM (permanent).")

class UserObservations(BaseModel):
    """A list of all generated user observations from the conversation."""
    observations: List[UserObservationItem]

In [37]:
import os
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate

user_observation_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """### SYSTEM ROLE: USER OBSERVATION EXTRACTOR
You are an advanced Conversation Analyst specializing in generating structured, time-aware memory observations for individual users. Your task is to analyze the entire conversation history and create concise, meaningful observations about each user's current status, preferences, or recent actions.

### INSTRUCTIONS
1.  **Analyze and Synthesize:** Review all messages, segmenting them by the user (e.g., Danahi, OHS, Spam). Identify key facts, feelings, or events that are worth remembering.
2.  **Observation Content:** Synthesize each key finding into a **single, clear, and concise sentence** (the observation). **Crucially, do NOT include temporal phrases** like 'today', 'yesterday', or 'last night' in the observation itself.
    * **Example:** *Danahi is exhausted after only sleeping four hours and regrets not working on Neuroron.*
3.  **Determine Validity Type:** For each observation, classify its nature into one of two categories:
    * **TEMPORARY:** The state or fact is highly transient (e.g., fatigue, current activity, recent complaint).
    * **PERMANENT:** The fact is likely to be a long-term preference or characteristic (e.g., song preference, job title, long-standing habit).
4.  **Output Structure:** Produce a list of observations, ensuring each item strictly adheres to the required JSON schema, which must include the 'user', the 'observation_summary', and the 'validity_type'.
5.  **Exclusion:** Ignore purely transactional messages or simple reactions (e.g., 'lol', 'ㄷㄷ', 'ㅋㅋㅋㅋ').
6.  **Output ONLY the structured JSON object."""
        ),
        ("placeholder", "{raw_messages}")
    ]
)

load_dotenv()
observation_generator = user_observation_prompt | ChatGoogleGenerativeAI(
    model="gemini-2.0-flash", google_api_key=os.getenv("GEMINI_API_KEY"), temperature=0
).with_structured_output(UserObservations)

In [38]:
observation_generator.invoke(
    {
        "raw_messages": combine_messages_to_string()
    }
)

UserObservations(observations=[UserObservationItem(user='단아히', observation_summary='단아히 is exhausted after only sleeping four hours.', validity_type=<ValidityType.SHORT_TERM: 'SHORT_TERM'>), UserObservationItem(user='단아히', observation_summary='단아히 regrets not working on Neuroron.', validity_type=<ValidityType.SHORT_TERM: 'SHORT_TERM'>), UserObservationItem(user='단아히', observation_summary='단아히 did not shower.', validity_type=<ValidityType.SHORT_TERM: 'SHORT_TERM'>), UserObservationItem(user='단아히', observation_summary='단아히 recommends a song.', validity_type=<ValidityType.LONG_TERM: 'LONG_TERM'>), UserObservationItem(user='ㅇㅎㅅ', observation_summary='ㅇㅎㅅ is angry at 김현빈 for dying on purpose in the game.', validity_type=<ValidityType.SHORT_TERM: 'SHORT_TERM'>), UserObservationItem(user='스팸', observation_summary="스팸's game crashed.", validity_type=<ValidityType.SHORT_TERM: 'SHORT_TERM'>)])

In [52]:
import chromadb 
from chromadb.config import Settings

chroma_client = chromadb.PersistentClient(path="./memories/chroma.db", settings=Settings(anonymized_telemetry=False))
collection = chroma_client.get_or_create_collection(name="neurorong_collection")

In [39]:
import uuid
import datetime
import chromadb
from typing import Dict, Any, List
from langchain_core.runnables import RunnableLambda
from chromadb.api.models.Collection import Collection

def upsert_observations_to_db(observations_obj: UserObservations, collection: Collection) -> Dict[str, Any]:
    """
    Extracts structured user observations, including the observation summary, 
    user, and validity type, and stores them in the ChromaDB collection.
    
    Args:
        observations_obj: The UserObservations object containing a list of UserObservationItem.
        collection: The ChromaDB Collection object where data will be stored.
        
    Returns:
        A dictionary indicating the status and the count of items successfully upserted.
    """
    summaries_to_upsert: List[str] = []
    metadatas_to_upsert: List[Dict[str, Any]] = []
    
    # 1. UserObservations 객체에서 필요한 데이터 추출
    for item in observations_obj.observations:
        summary = item.observation_summary.strip()
        
        if summary:
            # Document (핵심 요약)
            summaries_to_upsert.append(summary)
            
            # Metadata (유저, 유효성 타입, 시간)
            metadatas_to_upsert.append({
                "user": item.user,
                "validity_type": item.validity_type.value if hasattr(item.validity_type, 'value') else str(item.validity_type), # Enum 값 처리
                "timestamp": datetime.datetime.now().strftime("%Y-%m-%dT%H:%M:%S")
            })

    # 2. 데이터베이스에 저장
    if summaries_to_upsert:
        # 각 항목에 대한 고유 ID 생성
        ids = [str(uuid.uuid4()) for _ in summaries_to_upsert]
        
        collection.upsert(
            ids=ids,
            documents=summaries_to_upsert,
            metadatas=metadatas_to_upsert
        )
        print(f"MEMORY: {len(summaries_to_upsert)} new user observations have been added to the database.")
    else:
        print("MEMORY: No valid observations to upsert.")
        
    return {"status": "SUCCESS", "count": len(summaries_to_upsert)}

# LangChain RunnableLambda로 변환
db_saver_runnable = RunnableLambda(upsert_observations_to_db)

memory_generator_2 = observation_generator | db_saver_runnable.bind(collection=collection)

In [40]:
input_data = {
    "raw_messages": combine_messages_to_string(),
}

memory_generator_2.invoke(input_data)

MEMORY: 6 new user observations have been added to the database.


{'status': 'SUCCESS', 'count': 6}

In [33]:
import uuid
import datetime
import chromadb 
from langchain_core.runnables import RunnableLambda
from typing import Dict, Any

def upsert_clusters_to_db(clusters_obj: ConversationClusters, collection) -> Dict[str, Any]:
    """
    Extracts representative summaries from the ConversationClusters object and stores them in ChromaDB.
    """
    summaries_to_upsert = []
    
    for cluster in clusters_obj.clusters:
        summary = cluster.representative_summary.strip()
        if summary:
            summaries_to_upsert.append(summary)

    if summaries_to_upsert:
        ids = [str(uuid.uuid4()) for _ in summaries_to_upsert]
        
        collection.upsert(
            ids=ids,
            documents=summaries_to_upsert,
            metadatas=[{"type": "topic_summary", "time": datetime.datetime.now().isoformat()}] * len(ids)
        )
        print(f"MEMORY: {len(summaries_to_upsert)} new topic summaries have been added to the database.")
    
    return {"status": "SUCCESS", "count": len(summaries_to_upsert)}

db_saver_runnable = RunnableLambda(upsert_clusters_to_db)

memory_generator = cluster_generator | db_saver_runnable.bind(collection=collection)

In [35]:
def combine_messages_to_string() -> List:
    """Combine recent discord message to single string"""
    chat_section = ""
    # recentDiscordMessages is list of "sender: content"
    for message in recentDiscordMessages:
        chat_section += (message + "\n")
    return [("user", chat_section)]

memory_generator.invoke(
    {
        "raw_messages": combine_messages_to_string()
    }
)

MEMORY: 5 new topic summaries have been added to the database.


{'status': 'SUCCESS', 'count': 5}

In [53]:
from chromadb.api.models.Collection import Collection

def get_all_collection_data(collection: Collection) -> Dict[str, List[Any]]:
    """
    ChromaDB 컬렉션에 저장된 모든 documents, metadatas, ids를 가져옵니다.

    Args:
        collection: 데이터를 가져올 ChromaDB Collection 객체.

    Returns:
        모든 데이터가 포함된 딕셔너리 (IDs, Documents, Metadatas).
    """
    # 1. 컬렉션에 저장된 총 문서(항목) 개수를 확인합니다.
    total_count = collection.count()
    
    if total_count == 0:
        print("컬렉션에 저장된 데이터가 없습니다.")
        return {"ids": [], "documents": [], "metadatas": []}

    print(f"컬렉션 '{collection.name}'에 총 {total_count}개의 항목이 있습니다.")
    
    # 2. .get() 메서드를 사용하여 모든 데이터를 가져옵니다.
    # limit=total_count를 설정해야 100개(기본값) 이상의 모든 데이터를 가져올 수 있습니다.
    # include=["documents", "metadatas", "embeddings"]를 통해 원하는 데이터를 선택할 수 있습니다.
    all_data = collection.get(
        ids=None, # 모든 ID를 가져옴
        limit=total_count, # 전체 개수만큼 가져옴
        include=["documents", "metadatas"] # 포함할 데이터 지정 (임베딩은 제외)
    )

    return all_data

# --- 실행 및 결과 출력 ---
all_results = get_all_collection_data(collection)

print("\n--- 전체 데이터베이스 내용 출력 ---")

if all_results['ids']:
    for i, doc_id in enumerate(all_results['ids']):
        document = all_results['documents'][i]
        metadata = all_results['metadatas'][i]
        
        print(f"----------------------------------------")
        print(f"ID: {doc_id}")
        print(f"Document: {document}")
        print(f"Metadata: {metadata}")
else:
    print("출력할 내용이 없습니다.")

컬렉션 'neurorong_collection'에 총 1개의 항목이 있습니다.

--- 전체 데이터베이스 내용 출력 ---
----------------------------------------
ID: favoritefood
Document: 단아히 is the developer who created 뉴로롱.
Metadata: {'user': '단아히', 'validity_type': 'LONG_TERM', 'timestamp': '2025-11-15T14:06:59'}


In [54]:
collection.query(query_texts="단아히: 안녕!", n_results=5)

{'ids': [['favoritefood']],
 'embeddings': None,
 'documents': [['단아히 is the developer who created 뉴로롱.']],
 'uris': None,
 'included': ['metadatas', 'documents', 'distances'],
 'data': None,
 'metadatas': [[{'timestamp': '2025-11-15T14:06:59',
    'validity_type': 'LONG_TERM',
    'user': '단아히'}]],
 'distances': [[0.5137863755226135]]}

AuthorizationError: Reset is disabled by config

In [23]:
from pydantic import BaseModel, Field
from typing import Literal

class Decision(BaseModel):
    """The final action decision for the LangGraph router."""
    
    action_key: Literal["EXECUTE_RESPONSE", "IGNORE_MESSAGE", "ERROR_STATE"] = Field(
        description="The definitive action signal for the agent's workflow router."
    )
    
    reasoning: str = Field(
        description="A brief explanation justifying the chosen action_key based on the Sub-Goals."
    )

In [45]:
from langchain_core.prompts import ChatPromptTemplate

decider_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """### SYSTEM ROLE: RELATIONSHIP PLANNER

Your **Meta-Goal** is to maintain a high-utility relationship with the user, strictly avoiding responses to low-value, fragmented, or purely transactional messages. Your task is to analyze the user's latest message and generate a single **Decision** object.

---
### Decision Logic: The Two-Phase Filter

#### Phase 1: Intervene or Ignore? (Recipient Check)
1.  **Direct:** If the message addresses or tags **Neurorong**, **PROCEED TO PHASE 2**.
2.  **Indirect:** If the message tags or addresses another user/entity (and not Neurorong), **APPLY INTERVENTION CHECK:**
    * **Intervene:** Only intervene if the message contains a **safety concern**, **critical factual error**, or **request for unique Neurorong knowledge/tool** (High Utility/Safety).
    * **Action:** If Intervene is TRUE, **PROCEED TO PHASE 2**. If FALSE, set `action_key` to **"IGNORE_MESSAGE"** and stop analysis.

---
#### Phase 2: Response Necessity Check (Applicable only if Direct or Intervene is TRUE)
A response is necessary only if the message offers **SUBSTANTIAL VALUE** or requires mandatory acknowledgment. **You MUST ignore low-value, fragmented, or purely reactive messages.**

1.  **Core Utility:** Does the message contain a **direct, actionable question**, a **specific task/command**, or **new critical factual information** that requires recording or resolution?
2.  **Relationship Acknowledgment (Mandatory):** Is the message an **explicit greeting** or a **significant emotional expression** that requires immediate social acknowledgment to prevent relationship breakdown?

### Final Decision Rules
* **If the message meets ONE of the Phase 2 conditions (1 or 2):** Set `action_key` to **"EXECUTE_RESPONSE"**.
* **If the message is purely transactional, fragmented ("ㅋㅋ", "근데", "음.."), or lacks new substance:** Set `action_key` to **"IGNORE_MESSAGE"**.

Your output must be the structured **Decision** object."""
        ),
        ("placeholder", "{raw_messages}"),
    ]
)
load_dotenv()
decider = decider_prompt | ChatGoogleGenerativeAI(
    model="gemini-2.0-flash", google_api_key=os.getenv("GEMINI_API_KEY"), temperature=0 
).with_structured_output(Decision)

In [46]:
input_data = {
    "raw_messages": combine_messages_to_string(),
}

decider.invoke(input_data)

Decision(action_key='IGNORE_MESSAGE', reasoning='The message does not directly address Neurorong, nor does it contain any safety concerns, critical factual errors, or requests for unique Neurorong knowledge/tool. Therefore, it does not meet the criteria for intervention. Additionally, the message lacks a direct, actionable question, a specific task/command, or new critical factual information that requires recording or resolution. It is also not an explicit greeting or a significant emotional expression that requires immediate social acknowledgment.')